<a href="https://colab.research.google.com/github/Dhanraj7573/Python-Projects/blob/main/Amazon_Reviews_file_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
url = "http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz"
# Load first ~50k rows
chunk = next(pd.read_json(url, lines=True, compression="gzip", chunksize=50_000))
df = chunk.copy()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewerID      50000 non-null  object
 1   asin            50000 non-null  object
 2   reviewerName    49706 non-null  object
 3   helpful         50000 non-null  object
 4   reviewText      50000 non-null  object
 5   overall         50000 non-null  int64 
 6   summary         50000 non-null  object
 7   unixReviewTime  50000 non-null  int64 
 8   reviewTime      50000 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.4+ MB


In [17]:
from os import replace
import pandas as pd
import numpy as np
df.drop_duplicates(subset=["reviewerID", 'asin', 'reviewText'], inplace=True)
df['reviewText'] = df['reviewText'].fillna('').astype(str).str.strip()
df['summary'] = df['summary'].fillna('').astype(str).str.strip()
df["review_date"] = pd.to_datetime(df["unixReviewTime"], unit="s", errors="coerce")
df['review_clean'] = (
    df['reviewText']
    .str.lower()
    .str.replace(r"[^a-zA-Z0-9]", " ", regex=True)
    .str.replace(r"\s+"," ", regex=True)
    .str.strip()
)
df['review_len'] = df['review_clean'].str.len()
df['word_count'] = df['review_clean'].str.split().str.len()
df['sentiment'] = np.where(df['overall']>=4, 'positive',
                  np.where(df['overall']<=2, 'negative', 'neutral'))
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,review_date,review_clean,review_len,word_count,sentiment
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5,Gotta have GPS!,1370131200,"06 2, 2013",2013-06-02,we got this gps for my husband who is an otr o...,770,149,positive
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1,Very Disappointed,1290643200,"11 25, 2010",2010-11-25,i m a professional otr truck driver and i boug...,2113,443,negative
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3,1st impression,1283990400,"09 9, 2010",2010-09-09,well what can i say i ve had this unit in my t...,4490,870,neutral
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2,"Great grafics, POOR GPS",1290556800,"11 24, 2010",2010-11-24,not going to write a long review even thought ...,2208,463,negative
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1,"Major issues, only excuses for support",1317254400,"09 29, 2011",2011-09-29,i ve had mine for a year and here s what we go...,1056,210,negative
